In [4]:
import cv2
import glob
import os
import pickle as pkl

import numpy as np
import matplotlib.pyplot as plt
import statistics

from skimage import data, img_as_float, img_as_ubyte, io, color, img_as_uint
from skimage.measure import compare_ssim
from skimage.measure import compare_mse, compare_nrmse
from get_data_splits_newdata import get_model_data_splits
# %pylab inline

In [5]:
"""
python datasets/combine_A_and_B.py --fold_A /gpfs/data/lionnetlab/cellvision/pilotdata/20190314-pix2pix/channel1_1000/A --fold_B /gpfs/data/lionnetlab/cellvision/pilotdata/20190314-pix2pix/channel1_1000/B --fold_AB /gpfs/data/lionnetlab/cellvision/pilotdata/20190314-pix2pix/channel1_1000


bsub -gpu "num=1:mode=exclusive_process:mps=yes" -o log0314_channel1_1000_jobout4.txt python train.py --save_epoch_freq 50 --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20190314-pix2pix/channel1_1000 --name cellvision0314_channel1_1000 --model pix2pix --direction AtoB --gpu 0 --display_id 0 --loadSize 512 --fineSize 512


"""

'\npython datasets/combine_A_and_B.py --fold_A /gpfs/data/lionnetlab/cellvision/pilotdata/20190314-pix2pix/channel1_1000/A --fold_B /gpfs/data/lionnetlab/cellvision/pilotdata/20190314-pix2pix/channel1_1000/B --fold_AB /gpfs/data/lionnetlab/cellvision/pilotdata/20190314-pix2pix/channel1_1000\n\n\nbsub -gpu "num=1:mode=exclusive_process:mps=yes" -o log0314_channel1_1000_jobout4.txt python train.py --save_epoch_freq 50 --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20190314-pix2pix/channel1_1000 --name cellvision0314_channel1_1000 --model pix2pix --direction AtoB --gpu 0 --display_id 0 --loadSize 512 --fineSize 512\n\n\n'

In [10]:


source_imgs_path = '/gpfs/data/lionnetlab/cellvision/gfpdata/20190314-single_img'

channel4_img_paths = glob.glob('{}/amr_*_sample*_channel4.tif'.format(source_imgs_path))

comp_channels = ['channel1', 'channel2', 'channel3']

comp_channel_ssims = []
for comp_channel in  comp_channels:
    print("Performing {} comparison".format(comp_channel))
    channel_ssims = []
    for channel4_img_path in channel4_img_paths:
        channel4_img_path = channel4_img_path.replace('channel4', comp_channel)
        channel_img = io.imread(channel1_img_path).astype(np.uint)
        channel4_img = io.imread(channel4_img_path).astype(np.uint)
        channel_img_ft = img_as_float(channel_img)
        channel4_img_ft = img_as_float(channel4_img)
        _min = channel4_img_ft.min()
        _max = channel4_img_ft.max()
        channel_ssim = compare_ssim(channel_img_ft, channel4_img_ft, data_range=_max-_min)
        channel_ssims.append(channel_ssim)
    channel_mean = statistics.mean(channel_ssims)
    comp_channel_ssims.append( (comp_channel, channel_mean) )
print(comp_channel_ssims)


Performing channel1 comparison
Performing channel2 comparison
Performing channel3 comparison
[('channel1', 0.10344394259753834), ('channel2', 0.4300114707439371), ('channel3', 0.7234032725017239)]
